In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df_original = pd.read_csv('tmdb_movies_rawdata.csv')

In [3]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  object 
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  object 
 6   cast                  10790 non-null  object 
 7   homepage              2936 non-null   object 
 8   director              10822 non-null  object 
 9   tagline               8042 non-null   object 
 10  keywords              9373 non-null   object 
 11  overview              10862 non-null  object 
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  object 
 14  production_companies  9836 non-null   object 
 15  release_date       

In [4]:
#drop unwatend columns
droplist = ['id', 'imdb_id', 'budget', 'revenue', 'overview', 'tagline', 'homepage','keywords']
df=df_original.drop(droplist, axis=1)
#converting release date ot datetime format and adding a month column, quarter column
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_month_name'] = df['release_date'].dt.month_name()

quarters_numbers = [1,3,6,9,12,13]
quarters_names = ['winter','spring','summer','autumn','winter']
df['release_month_num'] = df['release_date'].dt.month
df['release_quarter'] = pd.cut(df['release_month_num'], bins=quarters_numbers, labels=quarters_names,ordered=False,right=False )

#adding profit-loss column
df['profit'] = df['revenue_adj']-df['budget_adj']
df['profit%'] = df['profit']/df['revenue_adj']*100
df.head(1)

,popularity,original_title,cast,director,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj,release_month_name,release_month_num,release_quarter,profit,profit%
0,32.985763,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,5562,6.5,2015,1.379999e+08,1.392446e+09,June,6,summer,1.254446e+09,90.089386


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   popularity            10866 non-null  float64       
 1   original_title        10866 non-null  object        
 2   cast                  10790 non-null  object        
 3   director              10822 non-null  object        
 4   runtime               10866 non-null  int64         
 5   genres                10843 non-null  object        
 6   production_companies  9836 non-null   object        
 7   release_date          10866 non-null  datetime64[ns]
 8   vote_count            10866 non-null  int64         
 9   vote_average          10866 non-null  float64       
 10  release_year          10866 non-null  int64         
 11  budget_adj            10866 non-null  float64       
 12  revenue_adj           10866 non-null  float64       
 13  release_month_na

In [6]:
#counting and dropping duplicated rows
df.duplicated().sum()

1

In [7]:
#getting the list of duplicated rows
df[df.duplicated()]

,popularity,original_title,cast,director,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj,release_month_name,release_month_num,release_quarter,profit,profit%
2090,0.59643,TEKKEN,Jon Foo|Kelly Overton|Cary-Hiroyuki Tagawa|Ian...,Dwight H. Little,92,Crime|Drama|Action|Thriller|Science Fiction,Namco|Light Song Films,2010-03-20,110,5.0,2010,30000000.0,967000.0,March,3,spring,-29033000.0,-3002.37849


In [8]:
#comfirming on duplication
df[df['original_title']=='TEKKEN']

,popularity,original_title,cast,director,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj,release_month_name,release_month_num,release_quarter,profit,profit%
2089,0.59643,TEKKEN,Jon Foo|Kelly Overton|Cary-Hiroyuki Tagawa|Ian...,Dwight H. Little,92,Crime|Drama|Action|Thriller|Science Fiction,Namco|Light Song Films,2010-03-20,110,5.0,2010,30000000.0,967000.0,March,3,spring,-29033000.0,-3002.37849
2090,0.59643,TEKKEN,Jon Foo|Kelly Overton|Cary-Hiroyuki Tagawa|Ian...,Dwight H. Little,92,Crime|Drama|Action|Thriller|Science Fiction,Namco|Light Song Films,2010-03-20,110,5.0,2010,30000000.0,967000.0,March,3,spring,-29033000.0,-3002.37849


In [9]:
df.drop_duplicates(inplace=True)

In [10]:
#I'll have a deeper look for null values before committing any actions.
#I'm sorting null values by order
df.isnull().sum().sort_values(ascending=False)

profit%                 4701
production_companies    1030
cast                      76
director                  44
genres                    23
profit                     0
original_title             0
runtime                    0
release_date               0
vote_count                 0
vote_average               0
release_year               0
budget_adj                 0
revenue_adj                0
release_month_name         0
release_month_num          0
release_quarter            0
popularity                 0
dtype: int64

In [11]:
"""since the total number of null values of cast, directors and genres combined equals(76+44+23)= 1.3% of all number of the dataset, then
it's okay to delete those null values as they won't affect severly on the sample.
"""
df = df.dropna(subset=['cast','director','genres'])
df.isnull().sum().sort_values(ascending=False)

#filling profit% nan with zero values 
df['profit%'] = df['profit%'].fillna(0)

In [12]:
#now combaring the weight of null rows of production companies against the whole sample
df.describe()

,popularity,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj,release_month_num,profit,profit%
count,10731.000000,10731.000000,10731.000000,10731.000000,10731.000000,1.073100e+04,1.073100e+04,10731.000000,1.073100e+04,1.073100e+04
mean,0.652615,102.468829,219.812972,5.964710,2001.259622,1.776530e+07,5.200623e+07,6.832448,3.424093e+07,-inf
std,1.004804,30.493873,578.815324,0.930283,12.820151,3.446630e+07,1.454252e+08,3.434309,1.259412e+08,NaN
min,0.000188,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00,1.000000,-4.139124e+08,-inf
25%,0.210765,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00,4.000000,0.000000e+00,0.000000e+00
50%,0.387081,99.000000,39.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00,7.000000,0.000000e+00,0.000000e+00
75%,0.720889,112.000000,148.000000,6.600000,2011.000000,2.110885e+07,3.470546e+07,10.000000,1.383952e+07,5.988608e+01
max,32.985763,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09,12.000000,2.750137e+09,1.000000e+02


In [13]:
df[df['production_companies'].isnull()].describe()

,popularity,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj,release_month_num,profit,profit%
count,959.000000,959.000000,959.000000,959.000000,959.000000,9.590000e+02,9.590000e+02,959.000000,9.590000e+02,959.0
mean,0.223562,97.803962,21.122002,5.976747,2005.143900,9.451630e+05,5.797874e+05,6.395203,-3.653756e+05,-inf
std,0.373217,49.644994,17.918650,1.089891,9.529327,4.445601e+06,4.546927e+06,3.639441,5.539130e+06,NaN
min,0.001117,0.000000,10.000000,2.100000,1961.000000,0.000000e+00,0.000000e+00,1.000000,-8.393668e+07,-inf
25%,0.088720,85.000000,12.000000,5.300000,2002.000000,0.000000e+00,0.000000e+00,3.000000,0.000000e+00,0.0
50%,0.173474,93.000000,15.000000,6.000000,2008.000000,0.000000e+00,0.000000e+00,7.000000,0.000000e+00,0.0
75%,0.287342,105.000000,23.000000,6.800000,2012.000000,0.000000e+00,0.000000e+00,10.000000,0.000000e+00,0.0
max,8.411577,900.000000,222.000000,9.200000,2015.000000,8.393668e+07,7.697178e+07,12.000000,6.775543e+07,100.0


In [14]:
df[df['production_companies'].isnull()].describe()/df.describe()*100

,popularity,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj,release_month_num,profit,profit%
count,8.936725,8.936725,8.936725,8.936725,8.936725,8.936725,8.936725,8.936725,8.936725,8.936725
mean,34.256340,95.447527,9.609079,100.201802,100.194092,5.320275,1.114842,93.600468,-1.067073,NaN
std,37.143247,162.803179,3.095746,117.156895,74.330854,12.898399,3.126644,105.973023,4.398188,NaN
min,594.148936,NaN,100.000000,140.000000,100.051020,NaN,NaN,100.000000,20.278849,NaN
25%,42.093939,94.444444,70.588235,98.148148,100.350877,NaN,NaN,75.000000,NaN,NaN
50%,44.815943,93.939394,38.461538,100.000000,100.099701,NaN,NaN,100.000000,NaN,NaN
75%,39.859299,93.750000,15.540541,103.030303,100.049727,0.000000,0.000000,100.000000,0.000000,0.000000
max,25.500629,100.000000,2.272960,100.000000,100.000000,19.749807,2.722618,100.000000,2.463711,100.000000


In [15]:
#so not to disturb the sample, we will gonna fill na production companies with unkown values
# I had an issue filling nan with 'unknown', so i had to convert the column into category type then adding unknown in category
# then replacing nan values with unknown
df['production_companies'] = df['production_companies'].astype("category")
df['production_companies'] = df['production_companies'].cat.add_categories("unknown").fillna("unknown")

In [16]:
df.head(2)

,popularity,original_title,cast,director,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj,release_month_name,release_month_num,release_quarter,profit,profit%
0,32.985763,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,5562,6.5,2015,1.379999e+08,1.392446e+09,June,6,summer,1.254446e+09,90.089386
1,28.419936,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,6185,7.1,2015,1.379999e+08,3.481613e+08,May,5,spring,2.101614e+08,60.363216


In [17]:
#this is our final cleaned dataset, where shall we start working on
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10731 entries, 0 to 10865
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   popularity            10731 non-null  float64       
 1   original_title        10731 non-null  object        
 2   cast                  10731 non-null  object        
 3   director              10731 non-null  object        
 4   runtime               10731 non-null  int64         
 5   genres                10731 non-null  object        
 6   production_companies  10731 non-null  category      
 7   release_date          10731 non-null  datetime64[ns]
 8   vote_count            10731 non-null  int64         
 9   vote_average          10731 non-null  float64       
 10  release_year          10731 non-null  int64         
 11  budget_adj            10731 non-null  float64       
 12  revenue_adj           10731 non-null  float64       
 13  release_month_na

In [18]:
#summary statistics of the dataset
df.describe()

,popularity,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj,release_month_num,profit,profit%
count,10731.000000,10731.000000,10731.000000,10731.000000,10731.000000,1.073100e+04,1.073100e+04,10731.000000,1.073100e+04,1.073100e+04
mean,0.652615,102.468829,219.812972,5.964710,2001.259622,1.776530e+07,5.200623e+07,6.832448,3.424093e+07,-inf
std,1.004804,30.493873,578.815324,0.930283,12.820151,3.446630e+07,1.454252e+08,3.434309,1.259412e+08,NaN
min,0.000188,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00,1.000000,-4.139124e+08,-inf
25%,0.210765,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00,4.000000,0.000000e+00,0.000000e+00
50%,0.387081,99.000000,39.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00,7.000000,0.000000e+00,0.000000e+00
75%,0.720889,112.000000,148.000000,6.600000,2011.000000,2.110885e+07,3.470546e+07,10.000000,1.383952e+07,5.988608e+01
max,32.985763,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09,12.000000,2.750137e+09,1.000000e+02


In [21]:
df.to_csv('cleared_data.csv', index=False)